# Part 2 - Constructing CSG geometry

This example builds on the previous model by adding a center column to the model.
The center column requires a different surface type.

OpenMC can use several different surface types in addition to spheres.
The documentation contains useful information all surface types supported.

https://openmc.readthedocs.io/en/stable/usersguide/geometry.html#surfaces-and-regions

For the center column we will used a cylinder with Z axis orientation.

In [ ]:
import openmc
import matplotlib.pyplot as plt

# surfaces
central_column_surface = openmc.ZCylinder(r=100) # note the new surface type
inner_sphere_surface = openmc.Sphere(r=480)
middle_sphere_surface = openmc.Sphere(r=500) 
outer_sphere_surface = openmc.Sphere(r=600, boundary_type='vacuum')

# regions
# the center column region is cut at the top and bottom using the -outer_sphere_surface
central_column_region = -central_column_surface & -outer_sphere_surface
firstwall_region = -middle_sphere_surface & +inner_sphere_surface & +central_column_surface
blanket_region = +middle_sphere_surface & -outer_sphere_surface & +central_column_surface
inner_vessel_region = +central_column_surface & -inner_sphere_surface

# cells
firstwall_cell = openmc.Cell(region=firstwall_region)
central_column_cell = openmc.Cell(region=central_column_region)
blanket_cell = openmc.Cell(region=blanket_region)
inner_vessel_cell = openmc.Cell(region=inner_vessel_region)

universe = openmc.Universe(cells=[central_column_cell, firstwall_cell,
                                  blanket_cell, inner_vessel_cell])

# VISULISATION
color_assignment = {blanket_cell: 'blue', firstwall_cell: 'red',
                    inner_vessel_cell:'grey', central_column_cell:'purple'}
plt.show(universe.plot(width=(1200, 1200), basis='xz', colors=color_assignment))
plt.show(universe.plot(width=(1200, 1200), basis='xy', colors=color_assignment))
plt.show(universe.plot(width=(1200, 1200), basis='yz', colors=color_assignment))


Another way to achieve the shape is to use Boolean operators.

In [ ]:
# surfaces
central_column_surface = openmc.ZCylinder(r=100) # note the new surface type
inner_sphere_surface = openmc.Sphere(r=480)
middle_sphere_surface = openmc.Sphere(r=500) 
outer_sphere_surface = openmc.Sphere(r=600, boundary_type='vacuum')

# regions, this time defined using boolean operators
# There are 3 opperators | OR, & AND, ~ NOT

central_column_region = -central_column_surface & -outer_sphere_surface

# the inner_vessel_region is defined using the logical NOT operator ~
inner_vessel_region = -inner_sphere_surface & ~central_column_region

# the firstwall_region is defined as below the middle surface and not in two other regions
firstwall_region = -middle_sphere_surface & ~inner_vessel_region & ~central_column_region

# the blanket_region is defined as between two surfaces and not the center_column_region
blanket_region = +middle_sphere_surface & -outer_sphere_surface & ~central_column_region


# cells defined in the same way
firstwall_cell = openmc.Cell(region=firstwall_region)
central_column_cell = openmc.Cell(region=central_column_region)
blanket_cell = openmc.Cell(region=blanket_region)
inner_vessel_cell = openmc.Cell(region=inner_vessel_region)

universe = openmc.Universe(cells=[central_column_cell, firstwall_cell,
                                  blanket_cell, inner_vessel_cell])

# VISULISATION
color_assignment = {blanket_cell: 'blue', firstwall_cell: 'red',
                    inner_vessel_cell:'grey', central_column_cell:'purple'}
plt.show(universe.plot(width=(1200, 1200), basis='xz', colors=color_assignment))
plt.show(universe.plot(width=(1200, 1200), basis='xy', colors=color_assignment))
plt.show(universe.plot(width=(1200, 1200), basis='yz', colors=color_assignment))


More details on CSG geometry construction methods can be found here:
https://docs.openmc.org/en/stable/usersguide/geometry.html

Additionally, OpenMC has a geometry viewing package which can be found here: https://github.com/openmc-dev/plotter

**Learning outcomes for Part 2:**

- Using different surface types in OpenMC to construct CSG.
- Defining geometry surfaces, regions and cells.